In [44]:
!pip install scikit-learn -q

In [57]:
!pip install torchsummaryX wandb --quiet

In [113]:
!pip install tqdm --quiet

In [1]:
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from torchsummaryX import summary
import wandb
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm


In [2]:
import torch

if torch.backends.mps.is_available():
    print("MPS backend is available!")
else:
    print("MPS backend is not available.")
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

MPS backend is available!


In [4]:
df = pd.read_csv("merged_output2.csv")

In [5]:
df = df.dropna()

In [6]:
na_rows = df[df.isna().any(axis=1)]

In [7]:
na_rows

,Unnamed: 0_x,Timestamp,People Count,RGB,Pico,Target


In [8]:
df

,Unnamed: 0_x,Timestamp,People Count,RGB,Pico,Target
0,6206.0,2025-05-02 11:28:08.000,4.0,4.0,5.0,5.0
1,6205.0,2025-05-02 11:28:07.000,3.0,4.0,5.0,4.0
2,6204.0,2025-05-02 11:28:06.000,3.0,4.0,5.0,4.0
3,6203.0,2025-05-02 11:28:05.000,2.0,3.0,5.0,4.0
4,6202.0,2025-05-02 11:28:03.000,2.0,3.0,5.0,4.0
...,...,...,...,...,...,...
6008,10.0,2025-04-28 16:04:14.000,4.0,4.0,4.0,4.0
6010,8.0,2025-04-28 16:04:12.000,4.0,5.0,4.0,4.0
6011,7.0,2025-04-28 16:04:11.000,4.0,4.0,4.0,4.0
6012,6.0,2025-04-28 16:04:10.000,5.0,4.0,4.0,4.0


In [47]:
x = df.iloc[:, 2:5]

In [48]:
y = df.iloc[:, -1]
y

0       5.0
1       4.0
2       4.0
3       4.0
4       4.0
       ... 
6008    4.0
6010    4.0
6011    4.0
6012    4.0
6013    4.0
Name: Target, Length: 5569, dtype: float64

# NN MLP

In [10]:
train, temp = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)


In [11]:
test.iloc[:, 2:5]

,People Count,RGB,Pico
160,2.0,3.0,4.0
1356,3.0,5.0,4.0
770,2.0,3.0,5.0
5596,3.0,3.0,0.0
1235,3.0,6.0,4.0
...,...,...,...
1433,3.0,5.0,4.0
5995,4.0,3.0,4.0
3114,1.0,3.0,0.0
1125,3.0,4.0,4.0


In [12]:
class OccupancyDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, index):
        return (torch.tensor(self.x.iloc[index].values, dtype=torch.float32),
                torch.tensor(self.y.iloc[index], dtype=torch.float32))


training = OccupancyDataset(train.iloc[:, 2:5],  train.iloc[:, -1])
validation = OccupancyDataset(val.iloc[:, 2:5],  val.iloc[:, -1])
test = OccupancyDataset(test.iloc[:, 2:5],  test.iloc[:, -1])

In [60]:
config = {
    'activations': 'GELU',
    'learning_rate': 0.001,
    'max_lr' : 0.004,
    'pct_start': 0.1,
    'optimizers': 'AdamW',
    'scheduler': 'OneCycleLR', #'ReduceLROnPlateau'
    'epochs': 25,
    'batch_size': 32,
    'weight_initialization': 'kaiming_normal', # e.g kaiming_normal, kaiming_uniform, uniform, xavier_normal or xavier_uniform
    'dropout': 0.2
 }

In [61]:
train_loader = torch.utils.data.DataLoader(
    dataset     = training,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True,
)


val_loader = torch.utils.data.DataLoader(
    dataset     = validation,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

In [62]:
all = []
for i, data in enumerate(val_loader):
    sensor_data, target = data
    all.append(target)
    print(len(sensor_data))
    break

32


In [63]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layers, dropout_rate):
        super().__init__()
        self.sequential = nn.ModuleList()

        #input layer
        self.sequential.append(nn.Linear(input_dim, hidden_dim))
        self.sequential.append(nn.ReLU())
        self.sequential.append(nn.BatchNorm1d(hidden_dim))

        #hidden
        for i in range(layers):
            self.sequential.append(nn.Linear(hidden_dim, hidden_dim))
            self.sequential.append(nn.ReLU())
            self.sequential.append(nn.BatchNorm1d(hidden_dim))

        #output layer
        self.sequential.append(nn.Linear(hidden_dim, output_dim))

    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                if config["weight_initialization"] == "xavier_normal":
                    torch.nn.init.xavier_normal_(m.weight)
                elif config["weight_initialization"] == "xavier_uniform":
                    torch.nn.init.xavier_uniform_(m.weight)
                elif config["weight_initialization"] == "kaiming_normal":
                    torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                elif config["weight_initialization"] == "kaiming_uniform":
                    torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                elif config["weight_initialization"] == "uniform":
                    torch.nn.init.uniform_(m.weight)
                else:
                    raise ValueError("Invalid weight_initialization value")
                m.bias.data.fill_(0)
    def forward(self, x):
        for layer in self.sequential:
            x = layer(x)
        return x

In [64]:
model = MLP(input_dim=3, 
            hidden_dim=25, 
            output_dim=1, 
            layers = 20, 
            dropout_rate= config["dropout"]).to(device)
summary(model, sensor_data.to(device))


----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                     [3, 25]             [32, 25]                 0.10                 0.00
1_ReLU                             -             [32, 25]                    -                    -
2_BatchNorm1d                   [25]             [32, 25]                 0.05                 0.00
3_Linear                    [25, 25]             [32, 25]                 0.65                 0.00
4_ReLU                             -             [32, 25]                    -                    -
5_BatchNorm1d                   [25]             [32, 25]                 0.05                 0.00
6_Linear                    [25, 25]             [32, 25]                 0.65                 0.00
7_ReLU                             -             [32, 25]                    -                    -

In [65]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])
total_steps = config["epochs"]*len(train_loader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                                                       max_lr = config['max_lr'], 
                                                       pct_start = config['pct_start'], 
                                                       steps_per_epoch=len(train_loader),
                                                       anneal_strategy = 'cos',
                                                       epochs=config["epochs"]
                                                       )
criterion = torch.nn.MSELoss()


In [70]:
wandb.login(

    key="f449e5715dbf82026aae85dffabb14116b5aa142"

)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [66]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    tloss = 0.0
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, desc="Training")
    for i, (input, target) in enumerate(dataloader):
        optimizer.zero_grad()

        input = input.to(device)
        target = target.to(device).unsqueeze(1)
        #print(target.shape)

        logits = model(input)
        #print(logits.shape)
        loss = criterion(logits, target)
        tloss+=loss.item()

        loss.backward()
        optimizer.step()
        

        batch_bar.update()
    scheduler.step()
    batch_bar.close()
    tloss /= len(train_loader)
    return tloss


In [67]:
def eval(model, dataloader):
    model.eval()
    vloss = 0.0
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Validation')
    with torch.no_grad():
        for i, (input, target) in enumerate(dataloader):
            optimizer.zero_grad()

            input = input.to(device)
            target = target.to(device).unsqueeze(1)
            

            logits = model(input)
            loss = criterion(logits, target)
            vloss+=loss.item()

            batch_bar.update()
            #print("logits shape:", logits.shape)
            #print("target shape:", target.shape)

    batch_bar.close()
    vloss /= len(val_loader)
    return vloss

In [69]:
wandb.finish()

BrokenPipeError: [Errno 32] Broken pipe

In [72]:
run = wandb.init(
    project="autonomous_project",  # Specify your project
    config = config,
    name = "run5"
)

BrokenPipeError: [Errno 32] Broken pipe

In [53]:
for epoch in range(config["epochs"]):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))
    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss   = train(model, train_loader, optimizer, criterion)
    val_loss       = eval(model, val_loader)

    print(f"Epoch {epoch}/20", flush=True)  # Ensure immediate flushing of output
    # Your training and validation logic here
    # Example:
    print(f"\tTrain Loss {train_loss:.04f}\t Learning Rate {curr_lr:.07f}")
    print(f"\tVal Loss {val_loss:.04f}")

    wandb.log({'train_loss': train_loss,
               'valid_loss': val_loss, 'lr': curr_lr})


Epoch 1/20


Training: 100%|██████████| 122/122 [00:05<00:00, 23.11it/s]
                                                            

Epoch 0/20


	Train Loss 12.0614	 Learning Rate 0.0002800
	Val Loss 11.6866

Epoch 2/20


Training: 100%|██████████| 122/122 [00:04<00:00, 25.48it/s]
                                                            

...

Epoch 4/20


	Train Loss 5.8276	 Learning Rate 0.0002845
	Val Loss 35.4160

Epoch 6/20


Training:  25%|██▍       | 30/122 [00:01<00:03, 25.68it/s]

KeyboardInterrupt: 

Training:  25%|██▍       | 30/122 [00:13<00:03, 25.68it/s]

In [142]:
target.shape

torch.Size([32])

In [143]:
sensor_data.shape

torch.Size([32, 3])